## Using Yahoo finance API to track current stocks

[Documentation for Yahoo Finance API](https://syncwith.com/yahoo-finance/yahoo-finance-api)

### Learning Objectives
1. Practice using Python to interact and call a public API sucessfully
2. Look at the data within JSON and DataFrames
3. Write data to local JSON file
4. Use basic vizualization to display information
5. Interact with users

### Assignment tasks
1. Takes user input for a stock (using Ticker Symbol): i.e. with the input() command
2. Display back to the user: ```Name Ticker```, ```Full name of the Stock```, ```Current Price```, ```Target Mean Price```, ```Cash on Hand```, and ```Profit Margins```.
3. Store results locally in JSON Format with just those items and include ```Date``` from when the data was pulled
4. Handle Errors (i.e. if the stock doesn't exist and/or the API is not returning information)

In [1]:
import os
import json
import pprint
import requests
import requests.exceptions
import pandas as pd

In [ ]:
# Getting user input for desired stock
stock = input()
print(f'The stock you have chosen is {stock}')

Setting in a base URL
(https://query1.finance.yahoo.com/v6/finance/quote)

In [ ]:
url = 'https://query1.finance.yahoo.com/v11/finance/quoteSummary/'
querystring = {"symbol": stock, "modules": 'financialData'} #type dict
querystring

In [ ]:
header_var ={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response = requests.request("GET",url, headers=header_var, params=querystring)
response.content

In [ ]:
stock_json = response.json()
stock_json

#### Variables included in financialData module
```Current Price```, ```Target Mean Price```, ```Cash on Hand```, and ```Profit Margins```.

In [ ]:
#longName = stock_json['quoteSummary']['result'][0]['financialData']
#ticker = stock_json['quoteSummary']['result'][0]['symbol']
#currentPrice = stock_json['quoteSummary']['result'][0]['currentPrice']['fmt']
current_price = stock_json['quoteSummary']['result'][0]['financialData']['currentPrice']['raw']
target_mean_price = stock_json['quoteSummary']['result'][0]['financialData']['targetMeanPrice']['raw']
cash_on_hand = stock_json['quoteSummary']['result'][0]['financialData']['totalCash']['raw']
profit_margin = stock_json['quoteSummary']['result'][0]['financialData']['profitMargins']['raw']


### Getting the name data from another API call

In [ ]:
url2 = 'https://query1.finance.yahoo.com/v7/finance/quote'
string_query = {"symbols": stock}
names = requests.request("GET", url2, headers=header_var, params=string_query)
name_json = names.json()
ticker_name = name_json['quoteResponse']['result'][0]['symbol']
full_name = name_json['quoteResponse']['result'][0]['longName']

### Adding responses to dataframe

In [ ]:
name_list = ['Ticker', 'Full Name', 'Current Price', 'Target Mean Price', 'Cash on Hand', 'Profit Margin']
value_list = [ticker_name, full_name,current_price, target_mean_price, cash_on_hand, profit_margin]
financial_data_dict = {'Attributes': name_list, "Values": value_list}
financial_data = {"Ticker": ticker_name, "Full Name": full_name, "Current Price": current_price, "Target Mean Price": target_mean_price, "Cash on Hand": cash_on_hand, "Profit Margins": profit_margin}
dataframe = pd.DataFrame.from_dict(financial_data_dict)
dataframe

## Modifying code to run in a function to handle expections

In [20]:
def get_api_response(stock, response_type):
    # Getting url for the financeData module in Yahoo API
    header_var ={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    finance_url = 'https://query1.finance.yahoo.com/v11/finance/quoteSummary/'
    finance_querystring = {"symbol": stock, "modules": 'financialData'} #type dict
    # Writing exceptions for if the url is not working
    try:
        finance_response = requests.request("GET", finance_url, headers=header_var, params=finance_querystring)
        finance_response.raise_for_status()
    except requests.exceptions.HTTPError as errh:
        return f"This stock '{stock}' does not exist: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "Connection" + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "Timeout error please try again: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "f4: " + repr(err)
    
    finance_json = finance_response.json()
    current_price = finance_json['quoteSummary']['result'][0]['financialData']['currentPrice']['raw']
    target_mean_price = finance_json['quoteSummary']['result'][0]['financialData']['targetMeanPrice']['raw']
    cash_on_hand = finance_json['quoteSummary']['result'][0]['financialData']['totalCash']['raw']
    profit_margin = finance_json['quoteSummary']['result'][0]['financialData']['profitMargins']['raw']
    
    # Getting the name data from Yahoo API
    name_url = 'https://query1.finance.yahoo.com/v7/finance/quote'
    name_querystring = {"symbols": stock}
    try:
        name_response = requests.request("GET", name_url, headers=header_var, params=name_querystring)
        name_response.raise_for_status()
    except requests.exceptions.HTTPError as errh:
        return f"This stock '{stock}' does not exist: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "Connection Error" + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "Time error please try again: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "4: " + repr(err)
    name_json = name_response.json()
    ticker_name = name_json['quoteResponse']['result'][0]['symbol']
    full_name = name_json['quoteResponse']['result'][0]['longName']
    name_list = ['Ticker', 'Full Name', 'Current Price', 'Target Mean Price', 'Cash on Hand', 'Profit Margin']
    value_list = [ticker_name, full_name,current_price, target_mean_price, cash_on_hand, profit_margin]
    data_dictionary = {'Attributes': name_list, "Values": value_list}
    json_dictionary = {
        "Ticker": ticker_name, 
        "Full Name": full_name, 
        "Current Price": current_price, 
        "Target Mean Price": target_mean_price, 
        "Cash on Hand": cash_on_hand, 
        "Profit Margins": profit_margin}


    if response_type == 'json':
        with open('lab04.json', 'w') as outfile:
            json.dump(json_dictionary, outfile)
        print("File written to json")
        result = json.dumps(json_dictionary, indent=4)
    elif response_type == 'dataframe':
        
        dataframe = pd.DataFrame.from_dict(data_dictionary)
        result = dataframe   
    else:
        result = "An unhandled error has occurred!"
        
    return result
stock = 'AAPL'
get_api_response(input(), 'json')

File written to json


'{\n    "Ticker": "MSFT",\n    "Full Name": "Microsoft Corporation",\n    "Current Price": 228.56,\n    "Target Mean Price": 329.65,\n    "Cash on Hand": 104748998656,\n    "Profit Margins": 0.36686\n}'

### Bonus: Plotting stock price history

In [84]:
import plotly.express as px
import plotly.graph_objects as go

**Intructions on using function**

For both interval and range the inputs that can be entered are:
- ```1m```: 1 minute
- ```2m```: 2 minutes
- ```5m```: 5 minutes
- ```15m```: 15 minutes
- ```30m```: 30 minutes
- ```60m```: 60 minutes
- ```90m```: 90 minutes
- ```1h```: 1 hour
- ```1d```: 1 day
- ```5d```: 5 days
- ```1wk```: 1 week
- ```1mo```: 1 month
- ```3mo```: 3 months


In [99]:
#https://query1.finance.yahoo.com/v8/finance/chart/
interval = '1m'
range = '5d'
def get_historical_data(interval, range):
    historical_url = 'https://query1.finance.yahoo.com/v8/finance/chart/'
    historical_querystring = {"symbol": "MSFT", "interval": interval, "range": range}
    historical = requests.request("GET", historical_url, headers=header_var, params=historical_querystring)
    historical_json = historical.json()
    return historical_json
historical_json = get_historical_data(interval, range)

In [102]:
high_list = historical_json['chart']['result'][0]['indicators']['quote'][0]['high']
import numpy as np
day_list = list(np.arange(1,len(high_list)+1))
high_dict = {"Day": day_list, "Price High": high_list }
high_df = pd.DataFrame.from_dict(high_dict)
px.line(high_df, 
        x = "Day",
        y="Price High",
        title=f"Line Plot for highest price of {stock} in the last {range}")

In [103]:
high_list = historical_json['chart']['result'][0]['indicators']['quote'][0]['high']
low_list = historical_json['chart']['result'][0]['indicators']['quote'][0]['low']
open_list = historical_json['chart']['result'][0]['indicators']['quote'][0]['open']
close_list = historical_json['chart']['result'][0]['indicators']['quote'][0]['close']
candle_dict = {
        "Day": day_list, 
        "Price Open": open_list, 
        "Price High": high_list,
        "Price Low": low_list,
        "Price Close": close_list}
candle_df = pd.DataFrame.from_dict(candle_dict)
fig = go.Figure(data=[go.Candlestick(x=candle_df['Day'],
                open=candle_df['Price Open'],
                high=candle_df['Price High'],
                low=candle_df['Price Low'],
                close=candle_df['Price Close'])
                ])
fig.update_layout(
    title=f'Historical Price over last {range}',
    yaxis_title=f'{stock} Price'
)
fig.show()